In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow
import cv2
import math

#mostly read file function
import numpy  
import pandas as pd
import glob
import numpy as np
import pickle
import joblib

from numpy import sqrt

from skimage.transform import pyramid_reduce, resize

#img function
import cv2 as cv
#thresholding filtering
import skimage.filters as filters

#ploting img or diagram function
import matplotlib.pyplot as plt

#train test split(split the dataset)
from sklearn.model_selection import train_test_split

#img preprocessing function
from sklearn.decomposition import PCA

#shuffle the list
from sklearn.utils import shuffle
                        
#to get the optimal parameter for the model
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

#models
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

#getting result tools
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import itertools
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA

from keras.callbacks import EarlyStopping, ModelCheckpoint

import tensorflow as tf
from tensorflow.keras import datasets, layers, models

import seaborn as sns
import matplotlib

%matplotlib inline

from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation

#preprocessing part
from skimage.segmentation import clear_border
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import ball, disk, binary_erosion, remove_small_objects, reconstruction, binary_closing, binary_opening
from skimage.filters import roberts, sobel
from scipy import ndimage as ndi

#plot 3d
from skimage import measure, feature
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [2]:
#get all classes names
classes = ["Bengin","Malignant","Normal"]

#get all images location(total 1097 img)
img_list = sorted(glob.glob('Dataset/*/*.*'))
print('Total: ', len(img_list))

#get all images location(total 1097 img)
# train_list = sorted(glob.glob('Dataset/Train/*.*'))
train_list = sorted(glob.glob('Dataset/Train/*.*'))
print(len(train_list))

#get all images location(total 1097 img)
# test_list = sorted(glob.glob('Dataset/Test/*.*'))
test_list = sorted(glob.glob('Dataset/Test/*.*'))
print(len(test_list))

Total:  42
30
12


In [3]:
#Define variable to hold X & y
#create numpy array placeholder for pixels with 1 channel(grayscale)
IMG_SIZE = 512
CHANNEL = 1
#arg: (length of numpy set, height, width, color channel)
X = np.empty((len(img_list), IMG_SIZE, IMG_SIZE), dtype=np.uint8)

y = []

# convert images to numpy arrays
for i, img_path in enumerate(img_list):
    # load image
    img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
    img = cv.resize(img, (IMG_SIZE, IMG_SIZE))
    X[i] = img
    y.append("Bengin") if img_path.find(classes[0]) != -1 else (y.append(classes[1]) if img_path.find(classes[1]) != -1 else y.append(classes[2]))
    
y = pd.Series(y)

# Segmented

In [4]:
def get_segmented_lungs(im, num, save=False, plot=False, show_on_window=False, crop_percentage=0.05):
    #This funtion segments the lungs from the given 2D slice.
    
    crop = im.copy()
    if show_on_window:
        height,width=im.shape[:2]
        start_row,start_col=int(height*crop_percentage),int(width*crop_percentage)
        end_row,end_col=int(height*(1-crop_percentage)),int(width*(1-crop_percentage))
        crop=crop[start_row:end_row,start_col:end_col]
    else:
        if num == 161 or (num >= 173 and num <= 174) or (num == 758):
            height,width=im.shape[:2]
            start_row,start_col=int(height*0.20),int(width*0.20)
            end_row,end_col=int(height*0.80),int(width*0.80)
            crop=crop[start_row:end_row,start_col:end_col]
        elif num >= 756 and num <= 767:
            #Step 1: Crop the image 
            height,width=im.shape[:2]
            start_row,start_col=int(height*0),int(width*0)
            end_row,end_col=int(height*1),int(width*1)
            crop=crop[start_row:end_row,start_col:end_col]
        elif num == 1320 or num == 1219 or (num >= 712 and num <= 767) or (num >= 779 and num <= 799) or (num >= 688 and num <= 699) or (num >= 648 and num <= 664) or (num >= 225 and num <= 234):
            #Step 1: Crop the image 
            height,width=im.shape[:2]
            start_row,start_col=int(height*0.03),int(width*0.03)
            end_row,end_col=int(height*0.97),int(width*0.97)
            crop=crop[start_row:end_row,start_col:end_col]
        else:
            #Step 1: Crop the image 
            height,width=im.shape[:2]
            start_row,start_col=int(height*0.12),int(width*0.12)
            end_row,end_col=int(height*0.88),int(width*0.88)
            crop=crop[start_row:end_row,start_col:end_col]
        
    #Step 2: Convert into a binary image. 
    ret,binary = cv.threshold(crop,140,255,cv.THRESH_BINARY_INV)
    
    #Step 3: Remove the blobs connected to the border of the image.
    cleared = clear_border(binary) 
    
    #Step 4: Closure operation with a disk of radius 10. This operation is 
    #to keep nodules attached to the lung wall.
    selem = disk(2)
    closing = binary_closing(cleared, selem)
        
    #Step 5: Label the image.
    label_image = label(closing)
    
    #Step 6: Keep the labels with 2 largest areas.
    areas = [r.area for r in regionprops(label_image)]
    areas.sort()
    if len(areas) > 2:
        for region in regionprops(label_image):
            if region.area < areas[-2]:
                for coordinates in region.coords:                
                       label_image[coordinates[0], coordinates[1]] = 0
    segmented_area = label_image > 0
        
    #Step 7: Erosion operation with a disk of radius 2. This operation is 
    #seperate the lung nodules attached to the blood vessels.
    selem = disk(2)
    erosion = binary_erosion(segmented_area, selem) 
        
    #     Step 4: Closure operation with a disk of radius 10. This operation is 
    #     to keep nodules attached to the lung wall.
    selem = disk(10)
    closing2 = binary_closing(erosion, selem)    
    
    #Step 8: Fill in the small holes inside the binary mask of lungs.
    edges = roberts(closing2)
    fill_holes = ndi.binary_fill_holes(edges)
        
    superimpose = crop.copy()
    #Step 9: Superimpose叠加 the binary mask on the input image.
    get_high_vals = fill_holes == 0
    superimpose[get_high_vals] = 0

    superimpose = cv.resize(superimpose, (512, 512))    
    
    if show_on_window:
        directory1 = 'result/'
        directory2 = '.jpg'
        images = [im, crop, binary, cleared, closing, segmented_area, erosion, closing2, fill_holes, superimpose]
        titles = ['0_original_image', '1_cropped_image', '2_binary_image', '3_remove_blobs', '4_closure', '5_roi', '6_erosion', '7_closure', '8_fill_hole', '9_result']
        for i, title in enumerate(titles):
            filename = directory1 + title + directory2
            try:
                cv.imwrite(filename, images[i])
            except:
                indices = images[i].astype(np.uint8)  #convert to an unsigned byte
                indices*=255
                cv.imwrite(filename, indices)
    else:
        #flip vertically
        directory1 = 'preprocessing/pre1/'
        directory2 = '.jpg'
        images = [crop, binary, cleared, label_image, superimpose]
        titles = ['cropped_image', 'binary_image', 'remove_blobs', 'label', 'result']

        if save:
            for y in range(5):
                filename = directory1 + str(y+1) + titles[y] + '/' + titles[y] + str(num+1) + directory2
                cv.imwrite(filename, images[y])

    images = [im, crop, binary, cleared, closing, label_image, segmented_area, erosion, closing2, fill_holes, superimpose]
    
    if plot:
        titles = ['Original Image', 
                 'Step 1: Cropped Image', 
                 'Step 2: Binary image', 
                 'Step 3: Remove blobs', 
                 'Step 4: Closure', 
                 'Step 5: Label', 
                 'Step 6: Region On Interest',
                 'Step 7: Erosion',
                 'Step 8: Closure', 
                 'Step 9: Fill Holes',
                 'Step 10: Result']
        plot_img(images, titles, camp=plt.cm.bone, rows = 3, cols = 4, fontsize= 50)
    
#     if show_on_window:
#         directory1 = 'result/'
#         directory2 = '.jpg'
#         titles = ['0_original_image', '1_cropped_image', '2_binary_image', '3_remove_blobs', '4_closure', '5_roi', '6_erosion', '7_fill_hole', '8_result']
#         for i, title in enumerate(titles):
#             filename = directory1 + title + directory2
#             try:
#                 cv.imwrite(filename, images[i])
#             except:
#                 indices = images[i].astype(np.uint8)  #convert to an unsigned byte
#                 indices*=255
#                 cv.imwrite(filename, indices)
    
    return superimpose

In [5]:
#Apply to all images
X_segmented = np.empty((len(img_list), IMG_SIZE, IMG_SIZE), dtype=np.uint8)

for i, img in enumerate(X):
    im = img.copy()
    X_segmented[i] = get_segmented_lungs(im, i)

# Get Nodules

In [6]:
#define function to get the nodules
def get_nodules_lungs(im, plot=False):
    
    #Step 1: Get Nodules in binary
    binary = im < 140
    
    get_high_vals = binary == 0
    
    nodules = im.copy()
    
    nodules[nodules < 140] = 0
    
    if plot == True:
        images = [im, binary, nodules]
        titles = ['Segmented Image', 'Binary Nodules Image', 'Nodules Image']
        plot_img(images, titles, rows = 1, cols = 3, fontsize= 10)
        
    return nodules

In [7]:
X_nodules = np.empty((len(img_list), IMG_SIZE, IMG_SIZE), dtype=np.uint8)

for i, img in enumerate(X_segmented):
    im = img.copy()
    X_nodules[i] = get_nodules_lungs(im)

# Get nodules further

In [8]:
# method 1: Remove based on the region properties
selem = ball(2)
binary = binary_closing(X_nodules, selem)

X_nodules_further = X_nodules.copy()

label_scan = label(binary)

areas = [r.area for r in regionprops(label_scan)]
areas.sort()

for r in regionprops(label_scan):
    max_x, max_y, max_z = 0, 0, 0
    min_x, min_y, min_z = 2000, 2000, 2000
    
    for c in r.coords:
        max_z = max(c[0], max_z)
        max_y = max(c[1], max_y)
        max_x = max(c[2], max_x)
        
        min_z = min(c[0], min_z)
        min_y = min(c[1], min_y)
        min_x = min(c[2], min_x)
    if (min_z == max_z or min_y == max_y or min_x == max_x or r.area > areas[-3]):
        for c in r.coords:
            X_nodules_further[c[0], c[1], c[2]] = 0
    else:
        index = (max((max_x - min_x), (max_y - min_y), (max_z - min_z))) / (min((max_x - min_x), (max_y - min_y) , (max_z - min_z)))

# Get nodules opening

In [9]:
# method 2: Using Opening method to remove
selem = ball(2)
X_nodules_opening = binary_closing(X_nodules, selem)

# Input image FFT

In [10]:
#Define variable to hold X & y
#create numpy array placeholder for pixels with 1 channel(grayscale)
IMG_SIZE = 512
# CHANNEL = 1
#arg: (length of numpy set, height, width, color channel)
X_segmented_fft = np.empty((len(img_list), IMG_SIZE, IMG_SIZE), dtype=np.uint8)
X_segmented_fft_imaginer = np.empty((len(img_list), IMG_SIZE, IMG_SIZE), dtype=np.uint8)
X_segmented_fft_abs = np.empty((len(img_list), IMG_SIZE, IMG_SIZE), dtype=np.uint8)

X_nodules_fft = np.empty((len(img_list), IMG_SIZE, IMG_SIZE), dtype=np.uint8)
X_nodules_fft_imaginer = np.empty((len(img_list), IMG_SIZE, IMG_SIZE), dtype=np.uint8)
X_nodules_fft_abs = np.empty((len(img_list), IMG_SIZE, IMG_SIZE), dtype=np.uint8)

X_nodules_further_fft = np.empty((len(img_list), IMG_SIZE, IMG_SIZE), dtype=np.uint8)
X_nodules_further_fft_imaginer = np.empty((len(img_list), IMG_SIZE, IMG_SIZE), dtype=np.uint8)
X_nodules_further_fft_abs = np.empty((len(img_list), IMG_SIZE, IMG_SIZE), dtype=np.uint8)

X_nodules_opening_fft = np.empty((len(img_list), IMG_SIZE, IMG_SIZE), dtype=np.uint8)
X_nodules_opening_fft_imaginer = np.empty((len(img_list), IMG_SIZE, IMG_SIZE), dtype=np.uint8)
X_nodules_opening_fft_abs = np.empty((len(img_list), IMG_SIZE, IMG_SIZE), dtype=np.uint8)

index = range(42)

for i, img_segmented,img_nodules, img_nodules_further,img_nodules_opening in zip(index, X_segmented, X_nodules, X_nodules_further, X_nodules_opening):
    img_segmented = np.fft.fft2(img_segmented)
    img_nodules = np.fft.fft2(img_nodules)
    img_nodules_further = np.fft.fft2(img_nodules_further)
    img_nodules_opening = np.fft.fft2(img_nodules_opening)
    
    X_segmented_fft[i] = img_segmented
    X_nodules_fft[i] = img_nodules
    X_nodules_further_fft[i] = img_nodules_further
    X_nodules_opening_fft[i] = img_nodules_opening
    
    
    X_segmented_fft_imaginer[i] = np.imag(img_segmented)
    X_nodules_fft_imaginer[i] = np.imag(img_nodules)
    X_nodules_further_fft_imaginer[i] = np.imag(img_nodules_further)
    X_nodules_opening_fft_imaginer[i] = np.imag(img_nodules_opening)
    
    X_segmented_fft_abs[i] = np.abs(img_segmented)
    X_nodules_fft_abs[i] = np.abs(img_nodules)
    X_nodules_further_fft_abs[i] = np.abs(img_nodules_further)
    X_nodules_opening_fft_abs[i] = np.abs(img_nodules_opening)

In [11]:
from tensorflow.keras.utils import to_categorical

r_rscv = y.copy()
y_rscv = pd.Series(r_rscv)
y_rscv = r_rscv.factorize()

# Convert y to one-hot encoding
y_categorical = to_categorical(y_rscv[0])

To convert the `y` array to categorical values using `tf.keras.utils.to_categorical`, you can use the following code:

```python
from tensorflow.keras.utils import to_categorical

# Convert y to one-hot encoding
y_categorical = to_categorical(y)
```

This will convert the `y` array into a one-hot encoded representation where each row represents a sample and each column represents a class. The number of columns in the resulting array will be equal to the number of unique classes in the `y` array.

# Training Segmented FFT

In [12]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.utils import to_categorical

# Load the data and labels
data = X_segmented_fft
labels = y_categorical

# Split the dataset into training, validation, and testing sets
train_data = data[:30]
train_labels = labels[:30]
val_data = data[30:36]
val_labels = labels[30:36]
test_data = data[36:]
test_labels = labels[36:]

# Normalize the pixel values to be between 0 and 1
train_data = train_data / 255.0
val_data = val_data / 255.0
test_data = test_data / 255.0

# # Convert the labels to categorical format
# train_labels = to_categorical(train_labels)
# val_labels = to_categorical(val_labels)
# test_labels = to_categorical(test_labels)

# Build the RNN model
model = Sequential()
model.add(LSTM(64, input_shape=(train_data.shape[1], train_data.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model on the training set and validate on the validation set
model.fit(train_data, train_labels, batch_size=8, epochs=50, validation_data=(val_data, val_labels), verbose=2)

# Evaluate the model on the testing set
test_loss, test_acc = model.evaluate(test_data, test_labels, verbose=2)
print('Test accuracy:', test_acc)

Epoch 1/50
4/4 - 4s - loss: 1.0954 - accuracy: 0.4667 - val_loss: 1.5847 - val_accuracy: 0.3333 - 4s/epoch - 924ms/step
Epoch 2/50
4/4 - 1s - loss: 1.1704 - accuracy: 0.3333 - val_loss: 1.5953 - val_accuracy: 0.3333 - 1s/epoch - 255ms/step
Epoch 3/50
4/4 - 1s - loss: 1.0948 - accuracy: 0.2333 - val_loss: 1.5790 - val_accuracy: 0.0000e+00 - 989ms/epoch - 247ms/step
Epoch 4/50
4/4 - 1s - loss: 1.0581 - accuracy: 0.4333 - val_loss: 1.6390 - val_accuracy: 0.0000e+00 - 1s/epoch - 272ms/step
Epoch 5/50
4/4 - 1s - loss: 1.0104 - accuracy: 0.5000 - val_loss: 1.6584 - val_accuracy: 0.0000e+00 - 1s/epoch - 255ms/step
Epoch 6/50
4/4 - 1s - loss: 1.0579 - accuracy: 0.4667 - val_loss: 1.7236 - val_accuracy: 0.0000e+00 - 1s/epoch - 251ms/step
Epoch 7/50
4/4 - 1s - loss: 1.0690 - accuracy: 0.4667 - val_loss: 1.7347 - val_accuracy: 0.0000e+00 - 1s/epoch - 251ms/step
Epoch 8/50
4/4 - 1s - loss: 1.0397 - accuracy: 0.4667 - val_loss: 1.7146 - val_accuracy: 0.0000e+00 - 989ms/epoch - 247ms/step
Epoch 9/50

# Training Nodules FFT

In [20]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.utils import to_categorical

# Load the data and labels
data = X_nodules_fft
labels = y_categorical

# Split the dataset into training, validation, and testing sets
train_data = data[:30]
train_labels = labels[:30]
val_data = data[30:36]
val_labels = labels[30:36]
test_data = data[36:]
test_labels = labels[36:]

# Normalize the pixel values to be between 0 and 1
train_data = train_data / 255.0
val_data = val_data / 255.0
test_data = test_data / 255.0

# Build the RNN model
model = Sequential()
model.add(LSTM(64, input_shape=(train_data.shape[1], train_data.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model on the training set and validate on the validation set
model.fit(train_data, train_labels, batch_size=8, epochs=50, validation_data=(val_data, val_labels), verbose=2)

# Evaluate the model on the testing set
test_loss, test_acc = model.evaluate(test_data, test_labels, verbose=2)
print('Test accuracy:', test_acc)

Epoch 1/50
4/4 - 4s - loss: 1.0519 - accuracy: 0.4000 - val_loss: 1.6611 - val_accuracy: 0.0000e+00 - 4s/epoch - 915ms/step
Epoch 2/50
4/4 - 1s - loss: 1.0453 - accuracy: 0.4000 - val_loss: 1.6315 - val_accuracy: 0.0000e+00 - 957ms/epoch - 239ms/step
Epoch 3/50
4/4 - 1s - loss: 0.9720 - accuracy: 0.5000 - val_loss: 1.6120 - val_accuracy: 0.0000e+00 - 948ms/epoch - 237ms/step
Epoch 4/50
4/4 - 1s - loss: 0.9420 - accuracy: 0.6333 - val_loss: 1.5579 - val_accuracy: 0.3333 - 930ms/epoch - 233ms/step
Epoch 5/50
4/4 - 1s - loss: 1.0214 - accuracy: 0.4667 - val_loss: 1.5398 - val_accuracy: 0.3333 - 936ms/epoch - 234ms/step
Epoch 6/50
4/4 - 1s - loss: 0.9867 - accuracy: 0.4333 - val_loss: 1.6229 - val_accuracy: 0.0000e+00 - 941ms/epoch - 235ms/step
Epoch 7/50
4/4 - 1s - loss: 0.9226 - accuracy: 0.6000 - val_loss: 1.6618 - val_accuracy: 0.0000e+00 - 957ms/epoch - 239ms/step
Epoch 8/50
4/4 - 1s - loss: 0.9817 - accuracy: 0.5667 - val_loss: 1.6871 - val_accuracy: 0.0000e+00 - 967ms/epoch - 242ms/

# Training Nodules Further FFT

In [21]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.utils import to_categorical

# Load the data and labels
data = X_nodules_further_fft
labels = y_categorical

# Split the dataset into training, validation, and testing sets
train_data = data[:30]
train_labels = labels[:30]
val_data = data[30:36]
val_labels = labels[30:36]
test_data = data[36:]
test_labels = labels[36:]

# Normalize the pixel values to be between 0 and 1
train_data = train_data / 255.0
val_data = val_data / 255.0
test_data = test_data / 255.0

# # Convert the labels to categorical format
# train_labels = to_categorical(train_labels)
# val_labels = to_categorical(val_labels)
# test_labels = to_categorical(test_labels)

# Build the RNN model
model = Sequential()
model.add(LSTM(64, input_shape=(train_data.shape[1], train_data.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model on the training set and validate on the validation set
model.fit(train_data, train_labels, batch_size=8, epochs=50, validation_data=(val_data, val_labels), verbose=2)

# Evaluate the model on the testing set
test_loss, test_acc = model.evaluate(test_data, test_labels, verbose=2)
print('Test accuracy:', test_acc)

Epoch 1/50
4/4 - 4s - loss: 1.1174 - accuracy: 0.3333 - val_loss: 1.6356 - val_accuracy: 0.0000e+00 - 4s/epoch - 1s/step
Epoch 2/50
4/4 - 1s - loss: 0.9896 - accuracy: 0.5000 - val_loss: 1.6080 - val_accuracy: 0.3333 - 1s/epoch - 263ms/step
Epoch 3/50
4/4 - 1s - loss: 0.9874 - accuracy: 0.4000 - val_loss: 1.6456 - val_accuracy: 0.3333 - 1s/epoch - 274ms/step
Epoch 4/50
4/4 - 1s - loss: 1.0027 - accuracy: 0.4667 - val_loss: 1.6039 - val_accuracy: 0.0000e+00 - 1s/epoch - 279ms/step
Epoch 5/50
4/4 - 1s - loss: 1.0201 - accuracy: 0.4333 - val_loss: 1.5591 - val_accuracy: 0.0000e+00 - 1s/epoch - 263ms/step
Epoch 6/50
4/4 - 1s - loss: 0.9422 - accuracy: 0.5333 - val_loss: 1.5253 - val_accuracy: 0.0000e+00 - 1s/epoch - 261ms/step
Epoch 7/50
4/4 - 1s - loss: 1.0541 - accuracy: 0.3667 - val_loss: 1.5375 - val_accuracy: 0.3333 - 1s/epoch - 263ms/step
Epoch 8/50
4/4 - 1s - loss: 0.9674 - accuracy: 0.5667 - val_loss: 1.5768 - val_accuracy: 0.0000e+00 - 1s/epoch - 267ms/step
Epoch 9/50
4/4 - 1s - l

# Training Nodules Opening FFT

In [22]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.utils import to_categorical

# Load the data and labels
data = X_nodules_opening_fft
labels = y_categorical

# Split the dataset into training, validation, and testing sets
train_data = data[:30]
train_labels = labels[:30]
val_data = data[30:36]
val_labels = labels[30:36]
test_data = data[36:]
test_labels = labels[36:]

# Normalize the pixel values to be between 0 and 1
train_data = train_data / 255.0
val_data = val_data / 255.0
test_data = test_data / 255.0

# # Convert the labels to categorical format
# train_labels = to_categorical(train_labels)
# val_labels = to_categorical(val_labels)
# test_labels = to_categorical(test_labels)

# Build the RNN model
model = Sequential()
model.add(LSTM(64, input_shape=(train_data.shape[1], train_data.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model on the training set and validate on the validation set
model.fit(train_data, train_labels, batch_size=8, epochs=50, validation_data=(val_data, val_labels), verbose=2)

# Evaluate the model on the testing set
test_loss, test_acc = model.evaluate(test_data, test_labels, verbose=2)
print('Test accuracy:', test_acc)

Epoch 1/50
4/4 - 4s - loss: 1.1286 - accuracy: 0.2000 - val_loss: 1.7969 - val_accuracy: 0.0000e+00 - 4s/epoch - 948ms/step
Epoch 2/50
4/4 - 1s - loss: 0.9789 - accuracy: 0.4667 - val_loss: 1.7450 - val_accuracy: 0.0000e+00 - 1s/epoch - 255ms/step
Epoch 3/50
4/4 - 1s - loss: 1.0193 - accuracy: 0.5333 - val_loss: 1.7179 - val_accuracy: 0.0000e+00 - 1s/epoch - 255ms/step
Epoch 4/50
4/4 - 1s - loss: 0.9311 - accuracy: 0.6667 - val_loss: 1.6316 - val_accuracy: 0.0000e+00 - 1s/epoch - 255ms/step
Epoch 5/50
4/4 - 1s - loss: 0.9761 - accuracy: 0.6000 - val_loss: 1.6151 - val_accuracy: 0.0000e+00 - 1s/epoch - 270ms/step
Epoch 6/50
4/4 - 1s - loss: 0.9558 - accuracy: 0.5333 - val_loss: 1.5342 - val_accuracy: 0.0000e+00 - 1s/epoch - 257ms/step
Epoch 7/50
4/4 - 1s - loss: 0.9560 - accuracy: 0.5333 - val_loss: 1.6363 - val_accuracy: 0.0000e+00 - 1s/epoch - 263ms/step
Epoch 8/50
4/4 - 1s - loss: 0.9633 - accuracy: 0.5333 - val_loss: 1.6239 - val_accuracy: 0.0000e+00 - 1s/epoch - 263ms/step
Epoch 9/

# Perform a gridsearch technique

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Load the data and labels
data = X_nodules_fft
labels = y_categorical

# Split the dataset into training, validation, and testing sets
train_data = data[:30]
train_labels = labels[:30]
val_data = data[30:36]
val_labels = labels[30:36]
test_data = data[36:]
test_labels = labels[36:]

# Normalize the pixel values to be between 0 and 1
train_data = train_data / 255.0
val_data = val_data / 255.0
test_data = test_data / 255.0

# Define the model
def create_model(neurons=64, lstm_layers=1, ann_layers=1, activation='softmax', dropout_layers=0, optimizer='adam'):
    model = Sequential()
    for i in range(lstm_layers):
        model.add(LSTM(neurons, input_shape=(train_data.shape[1], train_data.shape[2]), return_sequences=True if i < lstm_layers-1 else False))
        model.add(Dropout(0.2))
    for i in range(ann_layers):
        model.add(Dense(neurons, activation=activation))
        model.add(Dropout(0.2))
    for i in range(dropout_layers):
        model.add(Dropout(0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Define the hyperparameters grid
param_grid = {
    'neurons': [32, 64, 128],
    'lstm_layers': [1, 2, 3],
    'ann_layers': [1, 2, 3],
    'activation': ['relu', 'sigmoid', 'tanh'],
    'dropout_layers': [0, 1, 2],
    'optimizer': ['adam', 'rmsprop'],
    'batch_size': [8, 16, 32],
    'epochs': [50, 100],
    'verbose': [0]
}

# Define the grid search object
grid = GridSearchCV(estimator=KerasClassifier(build_fn=create_model), param_grid=param_grid, n_jobs=-1, scoring='accuracy')

# Fit the grid search object on the training set and validate on the validation set
grid_result = grid.fit(train_data, train_labels, validation_data=(val_data, val_labels))

# Print the best result
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Evaluate the model on the testing set
test_loss, test_acc = grid_result.best_estimator_.model.evaluate(test_data, test_labels, verbose=2)
print('Test accuracy:', test_acc)